# Patient Summary Generator

This notebook generates a comprehensive patient summary from EMBR format data (optimized FHIR data) using OpenAI's GPT model.

## Process
1. Loads environment variables including OpenAI API key
2. Reads patient data from .embr files
3. Processes the data using a customized prompt
4. Generates a structured summary including:
   - Demographics
   - Conditions
   - Medications
   - Care plans
   - Procedures
   - Key encounters and observations

The summary maintains strict adherence to the provided data without inference or external recommendations.

In [25]:
import openai
from pathlib import Path
from dotenv import load_dotenv
import os
from IPython.display import display, Markdown

# Load environment variables
load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

In [26]:
# Get first .embr file
embr_files = list(Path("../patients").glob("*.embr"))

# Arbitrary patient file, Adjust from 0-99 for different patients
with open(embr_files[11], 'r') as f:
    patient_data = f.read()

In [27]:

# Edit this prompt to fit your needs
prompt = f"""
You are analyzing patient data in EMBR format, an optimized summary of FHIR data. The data includes essential information.

Provide a summary with the following information:
Patient demographics
Conditions
Medications and MedicationRequests
Care plan and goals
Key procedures
Important Encounters and Observations


Only use the data provided, without adding, recommending, or inferring beyond it.
Reference any data with the dates and ID (e.g. EN-129 for Encounter 129).

Patient Data:
{patient_data}
"""

##### In this cell, we make an API call to OpenAI's GPT-4 model to analyze and summarize patient data. The model is given a system prompt identifying it as a healthcare professional, and a user prompt containing:
- The patient data in EMBR format
- Instructions to summarize specific aspects like demographics, conditions, medications etc.
- Guidelines to only use provided data and reference data points with dates/IDs

The response is stored in the 'summary' variable which contains the generated patient summary text.


In [32]:
response = openai.chat.completions.create(
    model="gpt-4o-mini", # Not all files can be summarized with gpt-4o, as the tokens are too high
    messages=[
        {"role": "system", "content": "You are a healthcare professional skilled at analyzing and summarizing patient records."},
        {"role": "user", "content": prompt}
    ],
    temperature=0.7,
    max_tokens=5000
)

summary = response.choices[0].message.content
display(Markdown(summary))


### Patient Summary

**Patient Demographics:**
- **Name:** Mr. Chas Clark Green
- **Gender:** Male
- **Date of Birth:** October 30, 1985
- **Address:** 244 Mohr Club Unit 36, Hudson, MA, 01749
- **Marital Status:** Never Married
- **Communication Language:** English (United States)

---

**Conditions:**
- **Confirmed Conditions:**
  - Risk activity involvement (EN-2, 2001-12-12)
  - Medication review due (EN-8, 2003-12-24)
  - Prediabetes (EN-15, 2008-01-02)
  - Stress (EN-15, 2008-01-02, resolved 2017-01-11)
  - Anemia (EN-26, 2011-01-05)
  - Acute viral pharyngitis (EN-42, 2016-01-08)
  - Viral sinusitis (EN-49, 2016-10-19)
  - Obesity (BMI 30+) (EN-58, 2016-10-26)
  - Gingivitis (EN-58, 2016-10-26, resolved 2016-11-16)
  - Sleep disorder (EN-101, 2016-11-02)
  - Sleep apnea (EN-139, 2016-12-05)
  - Essential hypertension (EN-155, 2017-01-11)
  - Metabolic syndrome X (EN-549, 2022-02-09)
  - Injury of knee (EN-666, 2023-06-19)
  - Rupture of patellar tendon (EN-680, 2023-06-26)
  - Injury of medial collateral ligament of knee (EN-778, 2024-07-10)

---

**Medications and Medication Requests:**
- **Active Medication:**
  - Sodium fluoride 0.0272 MG/MG Oral Gel (MD-122, active)
- **Medication Requests (Stopped):**
  - Amoxicillin 250 MG / Clavulanate 125 MG Oral Tablet (MR-51, 2016-10-19)
  - Hydrochlorothiazide 25 MG Oral Tablet (multiple requests, last on MR-765, active as of 2024-02-21)
  - Ibuprofen 200 MG Oral Tablet (MR-459, 2020-05-19)
  - Naproxen sodium 220 MG Oral Tablet (MR-670, 2023-06-19; MR-782, 2024-07-10)
  - Meperidine Hydrochloride 50 MG Oral Tablet (MR-683, 2023-06-26)
  - Acetaminophen 325 MG / HYDROcodone Bitartrate 7.5 MG Oral Tablet (MR-795, 2024-07-18)

---

**Care Plans and Goals:**
- **Diabetes Self-Management Plan:** 
  - Active since January 2, 2008 (CP-21).
- **Weight Reduction Regimen and CPAP Treatment for Sleep Apnea:**
  - Active since December 5, 2016 (CP-150).
- **Lifestyle Education Regarding Hypertension:** 
  - Active since January 11, 2017 (CP-208).
- **Wound Care for Laceration of Foot:**
  - Completed on May 19, 2020 (CP-463).
- **Physiotherapy Care Plan for Ruptured Patellar Tendon:**
  - Completed on June 19, 2023 (CP-674).
- **Physiotherapy Care Plan for Injury of Medial Collateral Ligament:**
  - Completed on July 10, 2024 (CP-786).

---

**Key Procedures:**
- **Sleep Apnea Assessments:**
  - Multiple assessments performed between November 2016 and February 2024.
- **Dental Procedures:**
  - Multiple dental consultations and care performed, including plaque removal and fluoride treatments from November 2016 to March 2023.
- **Emergency Room Admissions:**
  - Laceration treatment on May 19, 2020 (PR-458).
  - Knee injuries treated on June 19, 2023 (PR-669) and July 10, 2024 (PR-781).
- **Hospital Admissions for Joint Surgery:**
  - Non-urgent orthopedic admission for knee rupture on June 26, 2023 (PR-681).
  - Postoperative care for knee surgery on June 29, 2023 (PR-682).

---

**Important Encounters and Observations:**
- **Encounters:**
  - Regular general examinations and follow-ups with Dr. Rosa Caraballo and Dr. Araceli Willms from 2001 to 2024.
  
- **Observations:**
  - Vital signs and lab results consistently monitored, including weight, blood pressure, glucose levels, and hemoglobin A1c measurements at various encounters.
  - Notable blood pressure readings indicate hypertension with systolic values up to 151 mm(Hg) and diastolic values reaching 118 mm(Hg).

--- 

This summary reflects the patient's ongoing healthcare management, highlighting key health conditions, treatment plans, encounters, and significant laboratory observations.

# Analyzing Resource Usage

In the previous section, we generated a comprehensive patient summary by processing EMBR files and extracting relevant encounters and observations. Now, let's examine the resources and data processing steps required to build this summary. 

# Testability and Explainability
This approach ensures testability in your Generative AI results by verifying that the retrieved resources are accurate and free from hallucinations.

(This Code Will Update Obviously, Just Rough Implementation!)

In [96]:
def parse_embr_line(line):
    import re
    
    # Find all matches of key=value pattern
    matches = re.finditer(r'(\w+)=([^=]+?)(?=\s+\w+=|$)', line)
    
    # Convert matches to dictionary
    parsed = {}
    for match in matches:
        key = match.group(1)
        value = match.group(2).strip()
        parsed[key] = value
        
    return parsed

In [103]:
import re
from IPython.display import Markdown

# Find all instances of 2 letters followed by dash and numbers (e.g., EN-123, CO-456)
matches = re.findall(r'[A-Z]{2}-\d+', summary)

# Read the EMBR file and check first 10 chars for matches
with open(embr_files[11], 'r') as f:
    embr_content = f.read()
    embr_content = [line for line in embr_content.splitlines() if any(match in line[:10] for match in matches)]

formatted_lines = []

# Top-level heading for the summary
formatted_lines.append("**Resources In Summary:**")

for line in embr_content:
    result = parse_embr_line(line)

    # Add Observations with relevant information in indented bullet points
    formatted_lines.append("\n- **"+ ' '.join(line.split(' ')[0:2])+":**")
    if result.get('stat'):
        formatted_lines.append(f"  - Status: {result['stat']}")
    if result.get('date'):
        formatted_lines.append(f"  - Date: {result['date']}")
    if result.get('ref'):
        formatted_lines.append(f"  - References: {result['ref']}")
    
    # Additional details from other key-value pairs
    for key, value in result.items():
        if key not in ['ref', 'stat', 'date'] and value:
            # Format list-like values (like Codes) as comma-separated strings
            if isinstance(value, (list, set)):
                value = ', '.join(value)
            formatted_lines.append(f"  - {key.title()}: {value}")

# Add a concluding summary
formatted_lines.append("\nThis summary reflects the patient's ongoing healthcare management, highlighting key health conditions, treatment plans, encounters, and significant laboratory observations.")

# Join the formatted lines and display as markdown
display(Markdown('\n'.join(formatted_lines)))


**Resources In Summary:**

- **EN-2 Encounter:**
  - Status: finished
  - Date: 2001-12-12T15:08:17
  - References: {}
  - Codes: {Well child visit,ambulatory}
  - Provider: performer Dr. Rosa Caraballo
  - Facility: DR. HE'S PRIMARY CARE FOR ADULTS INC
  - Enddate: 2001-12-12T15:23:17

- **EN-8 Encounter:**
  - Status: finished
  - Date: 2003-12-24T15:08:17
  - References: {}
  - Codes: {General examination of patient,ambulatory}
  - Provider: performer Dr. Rosa Caraballo
  - Facility: DR. HE'S PRIMARY CARE FOR ADULTS INC
  - Enddate: 2003-12-24T16:00:30

- **EN-15 Encounter:**
  - Status: finished
  - Date: 2008-01-02T15:08:17
  - References: {}
  - Codes: {General examination of patient,ambulatory}
  - Provider: performer Dr. Rosa Caraballo
  - Facility: DR. HE'S PRIMARY CARE FOR ADULTS INC
  - Enddate: 2008-01-02T15:57:52

- **EN-26 Encounter:**
  - Status: finished
  - Date: 2011-01-05T15:08:17
  - References: {}
  - Codes: {General examination of patient,ambulatory}
  - Provider: performer Dr. Rosa Caraballo
  - Facility: DR. HE'S PRIMARY CARE FOR ADULTS INC
  - Enddate: 2011-01-05T15:47:33

- **EN-42 Encounter:**
  - Status: finished
  - Date: 2016-01-08T02:08:17
  - References: {}
  - Codes: {Encounter for symptom,Acute viral pharyngitis,ambulatory}
  - Provider: performer Dr. Araceli Willms
  - Facility: MARLBOROUGH HOSPITAL CORP
  - Enddate: 2016-01-08T02:23:17

- **EN-49 Encounter:**
  - Status: finished
  - Date: 2016-10-19T04:08:17
  - References: {}
  - Codes: {Encounter for symptom,Viral sinusitis,ambulatory}
  - Provider: performer Dr. Araceli Willms
  - Facility: MARLBOROUGH HOSPITAL CORP
  - Enddate: 2016-10-19T04:23:17

- **EN-58 Encounter:**
  - Status: finished
  - Date: 2016-10-26T16:08:17
  - References: {}
  - Codes: {Encounter for check up,ambulatory}
  - Provider: performer Dr. Araceli Willms
  - Facility: MARLBOROUGH HOSPITAL CORP
  - Enddate: 2016-10-26T16:41:24

- **EN-101 Encounter:**
  - Status: finished
  - Date: 2016-11-02T16:08:17
  - References: {}
  - Codes: {Encounter for symptom,Sleep disorder,ambulatory}
  - Provider: performer Dr. Araceli Willms
  - Facility: MARLBOROUGH HOSPITAL CORP
  - Enddate: 2016-11-02T16:59:02

- **EN-139 Encounter:**
  - Status: finished
  - Date: 2016-12-04T15:56:45
  - References: {}
  - Codes: {Follow-up visit,Sleep disorder,v3-ActCode IMP}
  - Provider: performer Dr. Araceli Willms
  - Facility: MARLBOROUGH HOSPITAL CORP
  - Enddate: 2016-12-05T15:56:45

- **EN-155 Encounter:**
  - Status: finished
  - Date: 2017-01-11T15:08:17
  - References: {}
  - Codes: {General examination of patient,ambulatory}
  - Provider: performer Dr. Rosa Caraballo
  - Facility: DR. HE'S PRIMARY CARE FOR ADULTS INC
  - Enddate: 2017-01-11T16:05:32

- **EN-213 Encounter:**
  - Status: finished
  - Date: 2017-01-25T15:08:17
  - References: {}
  - Codes: {Encounter for check up,Gingivitis,ambulatory}
  - Provider: performer Dr. Araceli Willms
  - Facility: MARLBOROUGH HOSPITAL CORP
  - Enddate: 2017-01-25T18:12:02

- **EN-227 Encounter:**
  - Status: finished
  - Date: 2017-02-10T15:08:17
  - References: {}
  - Codes: {Follow-up encounter,Essential hypertension,ambulatory}
  - Provider: performer Dr. Araceli Willms
  - Facility: MARLBOROUGH HOSPITAL CORP
  - Enddate: 2017-02-10T15:23:17

- **EN-233 Encounter:**
  - Status: finished
  - Date: 2017-04-11T16:08:17
  - References: {}
  - Codes: {Follow-up encounter,Essential hypertension,ambulatory}
  - Provider: performer Dr. Araceli Willms
  - Facility: MARLBOROUGH HOSPITAL CORP
  - Enddate: 2017-04-11T16:23:17

- **EN-239 Encounter:**
  - Status: finished
  - Date: 2017-06-10T16:08:17
  - References: {}
  - Codes: {Follow-up encounter,Essential hypertension,ambulatory}
  - Provider: performer Dr. Araceli Willms
  - Facility: MARLBOROUGH HOSPITAL CORP
  - Enddate: 2017-06-10T16:23:17

- **EN-245 Encounter:**
  - Status: finished
  - Date: 2018-01-17T15:08:17
  - References: {}
  - Codes: {General examination of patient,ambulatory}
  - Provider: performer Dr. Rosa Caraballo
  - Facility: DR. HE'S PRIMARY CARE FOR ADULTS INC
  - Enddate: 2018-01-17T16:03:57

- **EN-286 Encounter:**
  - Status: finished
  - Date: 2018-01-31T15:08:17
  - References: {}
  - Codes: {Encounter for check up,Patient referral for dental care,ambulatory}
  - Provider: performer Dr. Araceli Willms
  - Facility: MARLBOROUGH HOSPITAL CORP
  - Enddate: 2018-01-31T18:47:19

- **EN-427 Encounter:**
  - Status: finished
  - Date: 2020-02-12T15:08:17
  - References: {}
  - Codes: {Encounter for check up,Gingivitis,ambulatory}
  - Provider: performer Dr. Araceli Willms
  - Facility: MARLBOROUGH HOSPITAL CORP
  - Enddate: 2020-02-12T22:23:49

- **EN-549 Encounter:**
  - Status: finished
  - Date: 2022-02-09T15:08:17
  - References: {}
  - Codes: {General examination of patient,ambulatory}
  - Provider: performer Dr. Rosa Caraballo
  - Facility: DR. HE'S PRIMARY CARE FOR ADULTS INC
  - Enddate: 2022-02-09T16:02:47

- **EN-666 Encounter:**
  - Status: finished
  - Date: 2023-06-19T18:43:43
  - References: {}
  - Codes: {Emergency room admission,Injury of knee,emergency}
  - Provider: performer Dr. Gerry Davis
  - Facility: RESERVOIR CENTER FOR HEALTH & REHABILITATION, THE
  - Enddate: 2023-06-19T19:43:43

- **EN-680 Encounter:**
  - Status: finished
  - Date: 2023-06-26T18:43:43
  - References: {}
  - Codes: {Non-urgent orthopedic admission,Rupture of patellar tendon,v3-ActCode IMP}
  - Provider: performer Dr. Araceli Willms
  - Facility: MARLBOROUGH HOSPITAL CORP
  - Enddate: 2023-06-29T22:38:43

- **EN-778 Encounter:**
  - Status: finished
  - Date: 2024-07-10T22:38:43
  - References: {}
  - Codes: {Emergency room admission,Injury of knee,emergency}
  - Provider: performer Dr. Gerry Davis
  - Facility: RESERVOIR CENTER FOR HEALTH & REHABILITATION, THE
  - Enddate: 2024-07-10T23:38:43

- **EN-802 Encounter:**
  - Status: finished
  - Date: 2024-07-20T16:13:43
  - References: {}
  - Codes: {Hospital admission,Patient transfer to skilled nursing facility,v3-ActCode IMP}
  - Provider: performer Dr. Gerry Davis
  - Facility: RESERVOIR CENTER FOR HEALTH & REHABILITATION, THE
  - Enddate: 2024-07-29T16:28:43

- **CP-21 CarePlan:**
  - Status: active
  - Date: 2008-01-02T15:08:17
  - References: {EN-15,CN-16}
  - Codes: {Diabetic diet at DR. HE'S PRIMARY CARE FOR ADULTS INC,Exercise therapy at DR. HE'S PRIMARY CARE FOR ADULTS INC,careplan-category assess-plan,Diabetes self management plan,Diabetic diet,Exercise therapy}
  - Text: Care Plan for Diabetes self management plan.Care plan is meant to treat Prediabetes.Activities: Diabetes self management planDiabetes self management plan
  - Intent: order

- **CP-150 CarePlan:**
  - Status: active
  - Date: 2016-12-05T00:55:56
  - References: {EN-139,CN-140}
  - Codes: {Weight reduction regimen at MARLBOROUGH HOSPITAL CORP,Warning. Avoid alcoholic drink (qualifier value) at MARLBOROUGH HOSPITAL CORP,Continuous positive airway pressure ventilation treatment at MARLBOROUGH HOSPITAL CORP,careplan-category assess-plan,Care plan,Weight reduction regimen,Warning. Avoid alcoholic drink (qualifier value),Continuous positive airway pressure ventilation treatment}
  - Text: Care Plan for Care plan.Care plan is meant to treat Sleep apnea.Activities: Care planCare planCare plan
  - Intent: order

- **CP-208 CarePlan:**
  - Status: active
  - Date: 2017-01-11T15:08:17
  - References: {EN-155,CN-156}
  - Codes: {Prescribed activity/exercise education at DR. HE'S PRIMARY CARE FOR ADULTS INC,Counseling about alcohol consumption at DR. HE'S PRIMARY CARE FOR ADULTS INC,Dietary Approaches to Stop Hypertension diet at DR. HE'S PRIMARY CARE FOR ADULTS INC,Smoking cessation education at DR. HE'S PRIMARY CARE FOR ADULTS INC,careplan-category assess-plan,Lifestyle education regarding hypertension,Prescribed activity/exercise education,Counseling about alcohol consumption,Dietary Approaches to Stop Hypertension diet,Smoking cessation education}
  - Text: Care Plan for Lifestyle education regarding hypertension.Care plan is meant to treat Essential hypertension.Activities: Lifestyle education regarding hypertensionLifestyle education regarding hypertensionLifestyle education regarding hypertensionLifestyle education regarding hypertension
  - Intent: order

- **CP-463 CarePlan:**
  - Status: completed
  - Date: 2020-05-19T18:28:53
  - References: {EN-455,CN-457}
  - Codes: {Dressing change management at RESERVOIR CENTER FOR HEALTH & REHABILITATION, THE,Behavior to prevent infection (observable entity) at RESERVOIR CENTER FOR HEALTH & REHABILITATION, THE,careplan-category assess-plan,Wound care,Dressing change management,Behavior to prevent infection (observable entity)}
  - Text: Care Plan for Wound care.Care plan is meant to treat Laceration of foot.Activities: Wound careWound care
  - Intent: order
  - Enddate: 2020-06-08T18:43:43

- **CP-674 CarePlan:**
  - Status: completed
  - Date: 2023-06-19T18:43:43
  - References: {EN-666,CN-668}
  - Codes: {Rest, ice, compression and elevation treatment program at RESERVOIR CENTER FOR HEALTH & REHABILITATION, THE,Stretching exercises at RESERVOIR CENTER FOR HEALTH & REHABILITATION, THE,careplan-category assess-plan,Physiotherapy care plan,Rest, ice, compression and elevation treatment program,Stretching exercises}
  - Text: Care Plan for Physiotherapy care plan.Care plan is meant to treat Rupture of patellar tendon.Activities: Physiotherapy care planPhysiotherapy care plan
  - Intent: order
  - Enddate: 2023-09-07T22:38:43

- **CP-786 CarePlan:**
  - Status: completed
  - Date: 2024-07-10T22:38:43
  - References: {EN-778,CN-780}
  - Codes: {Rest, ice, compression and elevation treatment program at RESERVOIR CENTER FOR HEALTH & REHABILITATION, THE,Stretching exercises at RESERVOIR CENTER FOR HEALTH & REHABILITATION, THE,careplan-category assess-plan,Physiotherapy care plan,Rest, ice, compression and elevation treatment program,Stretching exercises}
  - Text: Care Plan for Physiotherapy care plan.Care plan is meant to treat Injury of medial collateral ligament of knee.Activities: Physiotherapy care planPhysiotherapy care plan
  - Intent: order
  - Enddate: 2024-09-18T16:13:43

- **MR-51 MedicationRequest:**
  - Status: stopped
  - Date: 2016-10-19T04:08:17
  - References: {EN-49,CN-50}
  - Codes: {Amoxicillin 250 MG / Clavulanate 125 MG Oral Tablet}
  - Intent: order
  - Requester: Dr. Araceli Willms

- **MR-459 MedicationRequest:**
  - Status: stopped
  - Date: 2020-05-19T18:43:43
  - References: {EN-455}
  - Codes: {Ibuprofen 200 MG Oral Tablet}
  - Intent: order
  - Requester: Dr. Gerry Davis
  - Dosageinstruction: Take as needed.

- **MR-670 MedicationRequest:**
  - Status: stopped
  - Date: 2023-06-19T18:43:43
  - References: {EN-666}
  - Codes: {Naproxen sodium 220 MG Oral Tablet}
  - Intent: order
  - Requester: Dr. Gerry Davis
  - Dosageinstruction: Take as needed.

- **MR-683 MedicationRequest:**
  - Status: stopped
  - Date: 2023-06-26T22:38:43
  - References: {EN-680}
  - Codes: {Meperidine Hydrochloride 50 MG Oral Tablet}
  - Intent: order
  - Requester: Dr. Araceli Willms
  - Dosageinstruction: 1 for 4.0 h

- **MR-765 MedicationRequest:**
  - Status: active
  - Date: 2024-02-21T15:08:17
  - References: {EN-721,CN-156}
  - Codes: {Hydrochlorothiazide 25 MG Oral Tablet}
  - Intent: order
  - Requester: Dr. Rosa Caraballo
  - Dosageinstruction: 1 for 1.0 d

- **MR-782 MedicationRequest:**
  - Status: stopped
  - Date: 2024-07-10T22:38:43
  - References: {EN-778}
  - Codes: {Naproxen sodium 220 MG Oral Tablet}
  - Intent: order
  - Requester: Dr. Gerry Davis
  - Dosageinstruction: Take as needed.

- **MR-795 MedicationRequest:**
  - Status: stopped
  - Date: 2024-07-18T02:13:43
  - References: {EN-792}
  - Codes: {Acetaminophen 325 MG / HYDROcodone Bitartrate 7.5 MG Oral Tablet}
  - Intent: order
  - Requester: Dr. Araceli Willms
  - Dosageinstruction: 1 for 6.0 h

- **PR-458 Procedure:**
  - Status: completed
  - Date: 2020-05-19T18:28:53
  - References: {EN-455,CN-457}
  - Codes: {Suture open wound}
  - Enddate: 2020-05-19T18:43:43

- **PR-669 Procedure:**
  - Status: completed
  - Date: 2023-06-19T18:43:43
  - References: {EN-666}
  - Codes: {Plain X-ray of knee region}
  - Enddate: 2023-06-19T19:13:43

- **PR-681 Procedure:**
  - Status: completed
  - Date: 2023-06-26T18:43:43
  - References: {EN-680,CN-668}
  - Codes: {Surgical manipulation of joint of knee}
  - Enddate: 2023-06-26T22:38:43

- **PR-682 Procedure:**
  - Status: completed
  - Date: 2023-06-26T22:38:43
  - References: {EN-680,CN-668}
  - Codes: {Postoperative care}
  - Enddate: 2023-06-29T22:38:43

- **PR-781 Procedure:**
  - Status: completed
  - Date: 2024-07-10T22:38:43
  - References: {EN-778}
  - Codes: {Plain X-ray of knee region}
  - Enddate: 2024-07-10T23:08:43

- **MD-122 Medication:**
  - Status: active
  - References: {}
  - Codes: {sodium fluoride 0.0272 MG/MG Oral Gel}

This summary reflects the patient's ongoing healthcare management, highlighting key health conditions, treatment plans, encounters, and significant laboratory observations.